## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [2]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [3]:
import yaml

In [4]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

/Users/gilles/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [6]:
saut="\n"

### Préparation des matrices de traits

In [7]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [8]:
sampleNumber="01-N"
genDigraphe=False
genGraphe=False
#samplePrefix="MGC-150916-extend-%s-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#samplePrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
samplePrefix="MGC-160104-%s-ext3-derivations-BRASS"%sampleNumber
#samplePrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
#samplePrefix="MGC-160104-derivation2-derives2"
#samplePrefix="MGC-160104-%s-basesDerives"%sampleNumber
sampleFile=samplePrefix+".csv"
#goldFile="MGC-160104-01-N-VerbesActions-GOLD.csv"
#goldFile="MGC-160104-01-N-Gold.csv"
#analysisPrefix="MGC-150916-extend-%s"%sampleNumber
#analysisPrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#analysisPrefix="MGC-160104-%s-ext3-paradigmes"%sampleNumber
analysisPrefix="MGC-160104-%s-ext3-derivations-BRASS"%sampleNumber
#analysisPrefix="MGC-160104-derivation2-%s-SILVER"%sampleNumber
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

In [9]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [10]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [11]:
cliqueFile="2015-Data/"+analysisPrefix+'-Network.pkl'
#paradigmes


In [12]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [13]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [14]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

### Définition des objets pour la gestion des règles

In [16]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [17]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [27]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys() if autre in ["FP"]]))
print (sorted(analyseCases))
#    analyseCases=sampleCases

[u'FP', u'FS', u'ai1P', u'ai1S', u'ai2P', u'ai2S', u'ai3P', u'ai3S', u'fi1P', u'fi1S', u'fi2P', u'fi2S', u'fi3P', u'fi3S', u'ii1P', u'ii1S', u'ii2P', u'ii2S', u'ii3P', u'ii3S', u'inf', u'is1P', u'is1S', u'is3P', u'is3S', u'pI1P', u'pI2P', u'pI2S', u'pP', u'pc1P', u'pc1S', u'pc2P', u'pc2S', u'pc3P', u'pc3S', u'pi1P', u'pi1S', u'pi2P', u'pi2S', u'pi3P', u'pi3S', u'ppFP', u'ppFS', u'ppMP', u'ppMS', u'ps1P', u'ps1S', u'ps2P', u'ps2S', u'ps3P', u'ps3S']


#### Calculer le score de la clique

In [41]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [42]:
basesDerives=pd.read_csv("2015-Data/MGC-160104-01-N-basesDerives.csv",sep=";",encoding="utf8")

In [43]:
with open("2015-Data/"+analysisPrefix+'-Network.pkl', 'rb') as input:
    infoCliques=pickle.load(input)
cliques=infoCliques["cliques"]
cliquesScores=infoCliques["cliquesScores"]
cliquesListes=infoCliques["cliquesListes"]

In [45]:
#cliques

In [46]:
listeTest=list(set(case[0].split("-")[0] for case in cliques))
len(listeTest)

195

In [47]:
analyseCases=list(set(case.split("-")[-1] for clique in cliques for case in clique ))
#sorted(analyseCases)

In [48]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

{2: 308,
 3: 589,
 4: 898,
 5: 1089,
 6: 1507,
 7: 1917,
 8: 1836,
 9: 1836,
 10: 1948,
 11: 1778,
 12: 1555,
 13: 1582,
 14: 1508,
 15: 1295,
 16: 1236,
 17: 1390,
 18: 1325,
 19: 1661,
 20: 1576,
 21: 1628,
 22: 1613,
 23: 1411,
 24: 1149,
 25: 1116,
 26: 925,
 27: 603,
 28: 408,
 29: 408,
 30: 256,
 31: 164,
 32: 73,
 33: 40,
 34: 40,
 35: 27,
 36: 10,
 37: 5,
 38: 9,
 39: 1,
 40: 3,
 41: 1,
 44: 1}

### Faire la liste des cases lexicalisées de l'échantillon

In [49]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])
#nbFormesLexicales

In [72]:
def cliques2Tableau(cliques):
    paradigmesListe=[]
    for n,clique in enumerate(cliques):
        paradigmeClique={}
        for element in clique:
            elements=element.split("-")
            lexeme=elements[0]
            nbInitial=len(casesLexicales[lexeme])
            forme=elements[-2]
            taminfo=elements[-1]
            paradigmeClique[taminfo]=forme
        paradigmeClique["lexeme"]=lexeme
        paradigmeClique["score"]=cliquesScores[n]
        paradigmeClique["ajouts"]=len(clique)-nbInitial
        paradigmesListe.append(paradigmeClique)
    return pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts"])


In [73]:
cliquesPANDAS=cliques2Tableau(cliques)

In [ ]:
filtreDERIVE=~(cliquesPANDAS["FS"].isnull() & cliquesPANDAS["FP"].isnull())
cliquesDERIVES=cliquesPANDAS[filtreDERIVE]

In [94]:
#cliquesDERIVES
for lexeme in listeTest:
    print (paradigmes[paradigmes["lexeme"]==lexeme].to_dict(orient="list")["pi3S"])
    for index,row in cliquesDERIVES[cliquesDERIVES["lexeme"]==lexeme].iterrows():
#        print (index,row)
        print (row["pi3S"])


[u's9ku']
s9ku
[u'r9pr\xe2']
r9prEn
r9prâ
nan
nan
nan
nan
nan
r9prEn
nan
nan
nan
nan
nan
nan
nan
nan
r9prâ
[u'pip']
pip
pip
nan
[u'\xe2t\xe2']
âtâ
nan
âtâd
nan
âtâ
nan
[u'syrpr\xe2']
syrprEn
nan
nan
syrprâ
nan
syrprâ
syrprâ
syrprEn
[u'\xe2kjEr']
nan
âkjEr
[u'pus']
nan
pus
pus
nan
[u'p\xf4']
pô
nan
nan
[u'asje,aswa']
nan
nan
nan
nan
nan
aswa
aswa
asje
asje
asje
[u'kri']
nan
nan
nan
nan
kri
kri
[u'pEz']
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
pEz
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
pEz
pEz
pEz
p9z
p9z
nan
nan
nan
nan
nan
nan
nan
nan
p9z
p9z
nan
nan
nan
nan
nan
p9z
nan
p9z
nan
nan
nan
p9z
p9z
nan
nan
nan
nan
nan
nan
nan
nan
p9z
p9z
p9z
p9z
nan
nan
nan
nan
p9z
p9z
nan
nan
nan
nan
nan
[u'dEkuvr']
nan
nan
nan
nan
dEkuvr
dEkuvr
[u'fl\xe2b']
nan
nan
nan
nan
[u't\xf4']
tô
nan
nan
[u'sOr']
sOrt
sOrt
sOrt
sO

## Comparer la sortie des cliques avec le paradigme de départ

In [23]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
progressBar = FloatProgress(min=0, max=len(cliques))
display(progressBar)
for n,clique in enumerate(cliques):
    progressBar.value=n
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        point=clique[0].split("-")
        lPoint=len(point)
#        print (point,"-".join(point[0:len(point)-2]))
        if lPoint==3:
            lexeme=point[0]
        else:
            lexeme="-".join(point[0:len(point)-2])
        paradigmeClique["lexeme"]=lexeme
#        if n%5000==0: print (n,int(100*float(n)/len(cliques)),end=", ")
#        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].notnull().columns.tolist()
#        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
#        casesLexeme.remove("lexeme")
        casesLexeme=casesLexicales[lexeme]
        nbInitial=len(casesLexeme)
        if casesLexeme and len(casesLexeme)<=len(clique):
            for element in casesLexeme:
#                print (lexeme,element, paradigmes[paradigmes["lexeme"]==lexeme][element])
                champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
                if ","  in champForme:
                    formes=champForme.split(",")
                    nbInitial+=len(formes)-1
                    okFormes=False
                    for forme in formes:
                        pointCase=u"%s-%s-%s"% (lexeme,forme,element)
#                        if debug: print (pointCase, clique)
                        if pointCase in clique:
                            okFormes=True
                            if debug: print ("point",pointCase)
#                            print ("clique",clique)
                            if debug: print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                    if okFormes:
                        sampleOK=True
                    else:
                        sampleOK=False
                        break
                else:
                    forme=champForme
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    if debug: print (pointCase, clique)
                    if not pointCase in clique:
                        sampleOK=False
#                        print ("point",pointCase)
#                        print ("clique",clique)
#                        print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                        break
        else:
            sampleOK=False
        if sampleOK:
            for element in clique:
                elements=element.split("-")
                forme=elements[-2]
                taminfo=elements[-1]
#                try:
#                 (lexeme,forme,taminfo)=element.split("-")
#                except ValueError:
#                    print (element)
                paradigmeClique[taminfo]=forme
#                if taminfo in paradigmes.columns:
#                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
#                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
#                            sampleOK=False
#                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
#            if n<100: print (clique==cliquesListes[n])
            paradigmeClique["score"]=cliquesScores[n]
            paradigmeClique["ajouts"]=len(clique)-nbInitial
#            print (cliqueScore(clique),clique)
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
#    else:
#        print ("break")
#        print (n,lexeme,clique)
#        break

#paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=["lexeme"]+analyseCases+[u"score",u"ajouts"])
#print (seuilClique)

CPU times: user 1min 8s, sys: 4.12 s, total: 1min 12s
Wall time: 1min 10s


In [24]:
filtreCLIQUES=~(paradigmesCLIQUES["FS"].isnull() & paradigmesCLIQUES["FP"].isnull())
paradigmesFILTRES=paradigmesCLIQUES[filtreCLIQUES]

In [25]:
paradigmeMAX=paradigmesFILTRES[paradigmesFILTRES["score"]==paradigmesFILTRES.groupby(["lexeme"])["score"].transform(max)].reset_index()
del paradigmeMAX["index"]
paradigmeSILVER=paradigmeMAX.groupby("lexeme").agg(lambda x: ",".join(list(set(x.dropna().values)))).reset_index()

In [26]:
paradigmeSILVER

,lexeme,FP,ii1P,ii1S,FS,is2S,is1S,pP,ai3S,ppMP,...,pc3S,pc3P,pc1P,pc1S,fi3S,fi3P,ai3P,fi1P,fi1S,pI1P
0,aller,,aljô,alE,ale,,,alâ,ala,ale,...,irE,irE,irjô,irE,ira,irô,alEr,irô,irE,alô
1,arriver,,arivjô,arivE,arive,,,arivâ,ariva,arive,...,ariv9rE,ariv9rE,ariv9rjô,ariv9rE,ariv9ra,ariv9rô,arivEr,ariv9rô,ariv9rE,arivô
2,astreindre,astrêt,,astrEJE,,,,astrEJâ,astrEJi,astrê,...,,,,,,astrêdrô,,,,
3,attendre,,atâdjô,atâdE,atât,,,atâdâ,atâdi,atâdy,...,atâdrE,atâdrE,,atâdrE,atâdra,atâdrô,atâdir,atâdrô,atâdrE,atâdô
4,bramer,,bramjô,bramE,brame,,,bramâ,brama,brame,...,,,,bram9rE,bram9ra,,bramEr,,bram9rE,bramô
5,branler,,brâljô,brâlE,brâle,,,brâlâ,brâla,brâle,...,brâl9rE,brâl9rE,,brâl9rE,brâl9ra,brâl9rô,brâlEr,,brâl9rE,brâlô
6,briser,,brizjô,brizE,brize,,,brizâ,briza,brize,...,briz9rE,briz9rE,,briz9rE,briz9ra,briz9rô,brizEr,briz9rô,briz9rE,brizô
7,buter,,bytjô,bytE,byte,,,bytâ,byta,byte,...,byt9rE,byt9rE,,byt9rE,byt9ra,byt9rô,bytEr,byt9rô,byt9rE,bytô
8,chevaucher,,,S9vOSE,S9vOSe,,,S9vOSâ,S9vOSa,S9vOSe,...,S9vOS9rE,,,,S9vOS9ra,S9vOS9rô,S9vOSEr,,S9vOS9rE,
9,choir,,,,Syt,,,,Sy,,...,,,,,,,,,,


In [27]:
paradigmeSILVER.to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-Derive.csv"),encoding="utf8",sep=";")
#GOLD.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Gold.csv',encoding="utf8",sep=";")

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
derivesFS=set(paradigmesCLIQUES[~(paradigmesCLIQUES["FS"].isnull())]["FS"].unique())
derivesFP=set(paradigmesCLIQUES[~(paradigmesCLIQUES["FP"].isnull())]["FP"].unique())
derivesFSFP=derivesFS|derivesFP
for n,derive in enumerate(sorted(derivesFSFP)):
    print (n,derive)

In [ ]:
paradigmesCLIQUES[~(paradigmesCLIQUES["FP"].isnull())]